In [79]:
import pandas as pd
import numpy as np
import os
import requests
import re

In [80]:
import warnings
warnings.filterwarnings('ignore')

# Concatenate monthly-end Billboard Hot 100

In [81]:
path = 'billboard_data/'
file = os.listdir(path)

year = [i for i in range(2012, 2022)]

file_name = sorted([f for f in file if f[:4] in str(year)])

hit = pd.read_csv(path + file_name[0])

for f in file_name[1:]:
    hit = pd.concat([hit,
                     pd.read_csv(path + f)],
                     ignore_index=True)
    
# add chart date for further reference 
## (i.e. if we want to denote songs ever featured top 20 as hit, chart date matters)
date = []

for i in file_name:
    date += [i[:-4]] *100
    
hit['date'] = date

hit

,artist,isNew,last,peak,rank,title,week,date
0,Adele,False,2,1,1,Set Fire To The Rain,21,2012-02-04
1,Rihanna Featuring Calvin Harris,False,1,1,2,We Found Love,18,2012-02-04
2,Flo Rida,False,3,3,3,Good Feeling,17,2012-02-04
3,David Guetta Featuring Nicki Minaj,False,10,4,4,Turn Me On,8,2012-02-04
4,Katy Perry,False,6,3,5,The One That Got Away,15,2012-02-04
...,...,...,...,...,...,...,...,...
11695,Kenny Chesney,False,100,87,96,Knowing You,6,2021-10-02
11696,Lil Nas X Featuring Miley Cyrus,True,0,97,97,Am I Dreaming,1,2021-10-02
11697,Zac Brown Band,True,0,98,98,Same Boat,1,2021-10-02
11698,H.E.R. Featuring Chris Brown,False,95,64,99,Come Through,15,2021-10-02


In [82]:
hit_unique = hit.drop_duplicates(subset=['title','artist'])[['title', 'artist','date']]
hit_unique['hit'] = [1]*len(hit_unique)
hit_unique.reset_index(inplace=True, drop=True)
hit_unique

,title,artist,date,hit
0,Set Fire To The Rain,Adele,2012-02-04,1
1,We Found Love,Rihanna Featuring Calvin Harris,2012-02-04,1
2,Good Feeling,Flo Rida,2012-02-04,1
3,Turn Me On,David Guetta Featuring Nicki Minaj,2012-02-04,1
4,The One That Got Away,Katy Perry,2012-02-04,1
...,...,...,...,...
3267,7am On Bridle Path,Drake,2021-10-02,1
3268,Off The Grid,Kanye West,2021-10-02,1
3269,In Da Getto,J Balvin & Skrillex,2021-10-02,1
3270,Am I Dreaming,Lil Nas X Featuring Miley Cyrus,2021-10-02,1


In [83]:
hit_unique['track_id'] = [np.nan] * len(hit_unique)
hit_unique['release_date'] = [np.nan] * len(hit_unique)
hit_unique['spotify.name'] = [np.nan] * len(hit_unique)
hit_unique['spotify.artists'] = [np.nan] * len(hit_unique)

## Get Spotify track ID

In [84]:
def getArtistNameList(result_artists):
    artists = []
    for a in result_artists:
        artists.append(a['name'])
    return artists

### Direect search by title and artists

In [85]:
CLIENT_ID = '6ced8cf12e0c470f9d9dd25a75d4ec26'
CLIENT_SECRET = '4ce366cebd7c42f2ac59bf96ca9f32b9'

AUTH_URL = 'https://accounts.spotify.com/api/token'

# POST
auth_response = requests.post(AUTH_URL, {
    'grant_type': 'client_credentials',
    'client_id': CLIENT_ID,
    'client_secret': CLIENT_SECRET,
})

# convert the response to JSON
auth_response_data = auth_response.json()

# save the access token
access_token = auth_response_data['access_token']
headers = {
    'Authorization': 'Bearer {token}'.format(token=access_token)
}

# base URL of all Spotify API endpoints
BASE_URL = 'https://api.spotify.com/v1/'

try:
    start_idx = hit_unique.loc[~hit_unique['track_id'].isnull(),:].index.values[-1] + 1
except:
    start_idx = 0
    
for i in range(start_idx, len(hit_unique)):
    # direct search by title and artist
    title = hit_unique['title'][i]
    artist = hit_unique['artist'][i]
    query = 'track:"%s" AND artist:%s&type=track&limit=1'%(title, artist)

    r = requests.get(BASE_URL + 'search?q=' + query, headers=headers).json()

    try:
        results = r['tracks']['items'][0]
        
        artists_i = getArtistNameList(results['artists'])

        for a in artists_i:
            # deal with middle name -> match only first and last name
            if (a.lower() in artist.lower()) or (artist.lower() in a.lower()) or\
            ((artist.split()[0].lower() in a.split()[0].lower()) and (artist.split()[-1].lower() in a.split()[-1].lower())):
                print(i, results['id'])
                hit_unique['track_id'][i] = results['id']
                hit_unique['release_date'][i] = results['album']['release_date']
                hit_unique['spotify.name'][i] = results['name']
                hit_unique['spotify.artists'][i] = artists_i
                break
    except:
        pass

0 73CMRj62VK8nUS4ezD2wvi
2 2LEF1A8DOZ9wRYikWgVlZ8
4 6hkOqJ5mE093AQf2lbZnsG
5 0RUGuh2uSNFJpGMSsD1F5C
6 0obBFrPYkSoBJbvHfUIhkv
8 4Li2WHPkuyCdtmokzW2007
9 6MAdEUilV2p9RQUqE5bMAK
10 0srwKuJPH8yBzzFUJMBQM2
14 2wAJTrFhCnQyNSD3oUgTZO
18 1zwMYTA5nlNjZxYrvBB2pV
19 6j7hih15xG2cdYwIJnQXsq
22 0laYHRpNTS6i8FXdupHkJ4
24 3Yt9lRtS5V4nbJnwcgFgvC
26 4YKAOXyqkKZ5gOZR0AmVMN
27 0UrheRSNBXBiZgudI7tVQ2
31 3RLC8jJmDOZwrJ5izaISXr
32 6LxSe8YmdPxy095Ux6znaQ
33 7uRznL3LcuazKpwCTpDltz
34 6nek1Nin9q48AVZcWs9e9D
36 28Sc21lrWcZD9Ov4rRkPJq
37 2OXidlnDThZR3zf36k6DVL
38 1IJYb2qBTHMoDzHgsuA86H
40 1c8gk2PeTE04A1pIDH9YMk
41 7w87IxuO7BDcJ3YUqCyMTT
42 1cktZnxiONH0hR6Q3DKy8h
43 1EQn3Uc5AyUXoiPLeyCrrg
44 0tr0D36L1RwPdUTFfyNq37
46 2khs94jHAVzUm3Pu4mAMmv
47 5YTtTbqxUzJHD3CpbNHerP
48 3hlksXnvbKogFdPbpO9vel
52 2Xdatbm43IXEVi0wmKeJve
53 6lanRgr6wXibZr8KgzXxBl
54 7B2ETOSXeXPfWQl4gLjcat
55 7gHXMSznrxRT2nZrYKMfDs
59 2atYprr9JsQaLynGhrWhld
60 2SEsRxNSXidnMOZoTEzX1k
61 54Y4vzwLZtkUnXeltlecq5
62 5B25Jpgu62IX8w1YtLibuA
63 3AGOgQzp0YcPH41u

479 6sy3LkhNFjJWlaeSMNwQ62
481 4cAgkb0ifwn0FSHGXnr4F6
483 4lcXNTm9JrQpJYXqsENmFI
484 722tgOgdIbNe3BEyLnejw4
487 1rIusB0mmJglKOFmV1iMft
488 5T6DM9qjjngWnukcw0svkX
489 5FDh0G5oQ2UD9JxWiDCT1x
490 2QDx0OJsKHrrjkHbfyP3pv
491 4CcrLWFQY4dRGMbUR59cSp
492 7bpvfjoneSQvW0tbtVm1Jk
493 52vS7mJ0a70Z4uRCfl8SjH
494 5ZMFVgZSYvbq3SmgRwaGMa
496 42GcjriRK6srwHkfbkBqVl
497 0eRdvRqJTPGZmdFftzF6Sx
498 5wOyNyVoLhGVEbRHf8W2pp
499 6GsAD8PgHxmEuIPTG8GP3M
502 64ujSykVY35bgU74PTmCsJ
503 2bvxIrwSsQiaHOePGrjnJ9
504 6Uy6K3KdmUdAfelUp0SeXn
507 0a4ZGSRZOgspQy2HYzU82l
508 6PUIzlqotEmPuBfjbwYWOB
509 0nrRP2bk19rLc0orkWPQk2
510 5FgLLtrDU4Ex2ezB9anpH5
511 1PNGJBI1Lx0h8QCpkHKZLU
512 1KIQ0RscHwxXPJUvhuO5Bl
513 2dLLR6qlu5UJ5gk0dKz0h3
515 6RjW45KHJ6kgI2xQ1aFa52
518 4HFNGbVmsMV5lHUvDuXLkl
519 6SjCa7LhEpPNiCxe3bLln4
521 5anCkDvJ17aznvK5TED5uo
522 0gL0Xf8CN95HmgfEYYLRZV
525 2QjOHCTQ1Jl3zawyYOpxh6
526 1qDahv56bbyCytqvnvY2Yp
527 0QTCTu0CXv4X1JEE4gNpGv
530 53Dj5PCDhb22qWqmre3YQs
531 0YKvEXArXE4GeGzqCsaifN
532 1yjY7rpaAQvKwpdUliHx0d
5

929 24tygl4LPQcQgeBe7qzY3n
930 6fv1RHhZIxsYbfI2atszgJ
932 69DR48pXzSabFUfQfDDJDb
933 5FVuERsYqtKKAaWe62NBnv
935 6AzCBeiDuUXGXjznBufswB
936 0tnGS4zZ9dvcHPLHidb9xr
939 6CkqeHWNY9gEjqQqcEnFF0
940 0lizgQ7Qw35od7CYaoMBZb
942 5Zb55VDys0H4jS8Me9I5aG
943 3Z3D2hnbZnveVhTuOifJyt
944 6rZ2PWCvrGET4tg03I5Mke
945 7D49Iig0avHre9RFSUMkd2
946 4PYiIgzcp4o5pEYji1xY9q
949 1WtdhlaiUoDTrVNSwaya0L
950 3WQOUr3vHqCHVheQir1xWj
951 4rUTAkOU7OkVGeQuyQCHhF
953 5AtswJe6AJ2uaqwHdVIsIT
954 4kbj5MwxO1bq9wjT5g9HaA
955 2bJvI42r8EF3wxjOuDav4r
957 1HNs07uZxZniPNmniw2GTa
958 2eQQ7YHhROyqSPiMrTOuum
959 0A5qqtFMG1qrNQEcjvr1A2
960 2j79NtwxoWDmkiH4MGwdLq
961 15WoYaFanAymIZuebfxvOt
962 6Ius4TC0L3cN74HT7ENE6e
963 2iuZJX9X9P0GKaE93xcPjk
964 5S9lfZu1YF8xKkgSsqL2Bd
965 3zHq9ouUJQFQRf3cm1rRLu
966 7FC9Lng8iXU081lbigu8m8
967 2kR18wsL3gvXRXdNE4dCGc
968 4lIxdJw6W3Fg4vUIYCB0S5
971 4frLb7nWtsz2ymBE6k2GRP
972 6DEaND0SHv3sC11xobZLiy
973 0k6DnZMLoEUH8NGD5zh2SE
974 5PUawWFG1oIS2NwEcyHaCr
976 4B1FxT0hP5dMWL64ubGR6P
978 19OKWQPIupVomZOgBHvhV3
9

1337 4ly1QBXEwYoDmje9rmEgC4
1338 2Z8WuEywRWYTKe1NybPQEW
1344 19a3JfW8BQwqHWUMbcqSx8
1345 4KW1lqgSr8TKrvBII0Brf8
1346 2bocAPQcUTLUXx8lAOTiK3
1347 0dbzWSYpMcRtwjI1S7Pkql
1348 3d8y0t70g7hw2FOWl9Z4Fm
1349 61QSuw5VlC0LTS8WMO356g
1351 2H8jWuNmluerTEsuh8Gu6y
1354 7zBPzAjKAqQpcv8F8GCq5s
1355 0UHB9METy4VCXNgkcGqHqS
1357 3nAq2hCr1oWsIU54tS98pL
1358 5htB2gxndGHrLb09x1Q3Vp
1359 1eQBEelI2NCy7AUTerX0KS
1360 7gHs73wELdeycvS48JfIos
1362 4llYMkhHa1w7UAdDFRQcEZ
1364 0XN2BcoaKE4Z2hnq1ZeDY1
1365 5fSPbm5lcwtqwXkeQQswW8
1370 0BCy325UZyR9z0t0uxwn2N
1371 3O8NlPh2LByMU9lSRSHedm
1372 7K5dzhGda2vRTaAWYI3hrb
1376 5NFYuqu8V6QXc6mzcLImd6
1377 3KwwE4sgCzMaKWq6QBebmX
1379 433P7tDcIAi6NLnf4Sh6tI
1381 3NxAG2ni1lLa8RKL6a0INc
1383 1C7KSXR2GVxknex6I4ANco
1384 3cjF2OFRmip8spwZYQRKxP
1385 6BbINUfGabVyiNFJpQXn3x
1387 7kftK4n15rl1wbpfjSq2Tk
1388 03L2AoiRbWhvt7BDMx1jUB
1390 13wYXGimJ5fANFu0y2pqG1
1395 67citk3uzWs5qbaIVKTeg8
1396 4cRBqWBjuccCowYVHFlXK6
1397 4PA16FAl8LPmFmOhARawdV
1398 7sBwAWyXfiIgrYQ8BaJESH
1403 1LWgoxKUgZIeodl

1789 19yTqpHTUrw7mOcRDvXYzn
1790 6xL9MJ2yt39HPPZPyGkRy8
1792 09iyGiljLWRYXdRazFkdtJ
1793 0s6OWiWGuiCcm1Muzch5A8
1794 1P17dC1amhFzptugyAO7Il
1795 3VlbOrM6nYPprVvzBZllE5
1796 2yLa0QULdQr0qAIvVwN6B5
1799 7iNIg7XDEaYECfWD5dJ9Va
1800 7m9OqQk4RVRkw9JJdeAw96
1805 1WIZiOuNO3woKfdlSK2gNn
1806 16QN8cBubEq706vNtPpOME
1808 1uL78kFWxhz3umTMWJu8n5
1809 7y9iMe8SOB6z3NoHE2OfXl
1811 3GVkPk8mqxz0itaAriG1L7
1812 4iLeL1aYGcQBsYcWIhXIUO
1813 43ZyHQITOjhciSUUNPVRHc
1814 2yQZwi1P8AkkxxFhQ8rMEK
1816 6SUxrwVN19Kmd9JfIbjJUj
1818 3ojTJaonfkL96iIWa47SU3
1819 5w3NgfV8v1nNZFLUBlGHIL
1821 5Gu0PDLN4YJeW75PpBSg9p
1824 75ZvA4QfFiZvzhj2xkaWAh
1825 6wmAHw1szh5RCKSRjiXhPe
1828 5yuShbu70mtHXY0yLzCQLQ
1829 3T6MAweakkNrMYs8jZIWtg
1830 6waqLPcT6ruMhmd36OhmFO
1832 3I4WFddKpe2PGZWFh7f10J
1834 52okn5MNA47tk87PeZJLEL
1837 31x4wWBghN7f5PiEZUmcAW
1838 52ioLaTnO2J7BWY8UN2n9D
1840 5jZF1nvKORdaHNjNNMRbFl
1842 2iXuNzdMklGtLwaI6o345H
1844 4wFjTWCunQFKtukqrNijEt
1846 7iDa6hUg2VgEL1o1HjmfBn
1847 7sO5G9EABYOXQKNPNiE9NR
1848 5u6vkDnOyaf8Lst

2218 3f5lA8nnc4YYF1qf78WpHw
2219 1XJBDDpeuZaj5fvmwhdIMw
2220 7sTtHHrD0zDpmzQzH3zegz
2221 3pJkBPHadRAxBBo484RipI
2222 3NgcaTIgVh77DdwAInp996
2223 6viHvc13l9A6Ua6xugpeX6
2225 68JS5SFTnW5Yv9Vzw81Jf0
2226 7hDVYcQq6MxkdJGweuCtl9
2228 7K0lR5Ngf5jkOWt1g6ut6l
2229 0TkJIyL69RAUIbNwGub6uR
2230 1dtIaSlyrLI04sqYa8nLyN
2231 2uFaJJtFpPDc5Pa95XzTvg
2232 5ASM6Qjiav2xPe7gRkQMsQ
2233 25leEEaz1gIpp7o21Fqyjo
2234 0hvN2v6fAcB6xWyW7UaooA
2235 4so0Wek9Ig1p6CRCHuINwW
2236 2pXpURmn6zC5ZYDMms6fwa
2238 2t8yVaLvJ0RenpXUIAC52d
2239 3QiAAp20rPC3dcAtKtMaqQ
2241 46pF1zFimM582ss1PrMy68
2242 2pnPe4pJtq7689i5ydzvJJ
2243 7xQAfvXzm3AkraOtGPWIZg
2244 2TMzojGlbvke4ei3lEvt27
2245 0CGdHeagmWijXcimOUXfcM
2246 0C2AvaDXgN5SHi3YOTsV5E
2250 3Ol2xnObFdKV9pmRD2t9x8
2251 4paAidoE8VAemwU4jgCc6l
2252 5ZSIExfQuVv69tw5Qw3yDl
2253 39LmTF9RgyakzSYX8txrow
2254 2FUNBaa5DwItJtYEBgAblU
2255 2MShy1GSSgbmGUxADNIao5
2257 2rxQMGVafnNaRaXlRMWPde
2258 7DnAm9FOTWE3cUvso43HhI
2259 01jIO8SJFnpSiNLH1JaBZ5
2260 2wOXxtHZgRkkrkEbKLzzqs
2261 4B8vU5oDXVxO5Ok

2622 2Yv2mHzr5AQavVdwQjEokV
2623 7ccTcabbJlCJiIqtrSSwBk
2624 3R8CyhJfVjvgIROd5RSGhQ
2625 7Dj2wqPYXfSRFwa6qyOztr
2627 3Qa944OTMZkg8DHjET8JQv
2628 5M8goiFYynmNxhueYW6grR
2629 6U0FIYXCQ3TGrk4tFpLrEA
2630 1OuN92HcVG6NVpWbeESNB3
2631 7HMmFQsKsljwTw8bS7lu19
2633 34d7sGX6WUqq8f04e7CSNF
2636 60RFlt48hm0l4Fu0JoccOl
2637 7Dxes6VnfbZYV0S2QTnVuT
2638 0AYbVoeF03NYhM0fvKjAuU
2640 5M3v6frAiTyqenNAeQVcxC
2641 1kwnxJNVl7cwcU98RvMBaR
2643 3QVHSV8YKYq4L8tI5rnFgj
2644 0Al3wN3keprGTSgG8NBwjr
2645 0udOCZp7yk0Ml92C0k8H8D
2647 73SpzrcaHk0RQPFP73vqVR
2650 2ZTYlnhhV1UAReg7wIGolx
2651 6ZelF5APDN5r6XnFqcnvWR
2652 79s5XnCN4TJKTVMSmOx8Ep
2653 6wJYhPfqk3KGhHRG76WzOh
2654 6cZH4rX1KTt1aJ3Ql6Ynja
2657 5MwynWK9s4hlyKHqhkNn4A
2658 6MBUUSIWCzaXW4q58Ktrv9
2659 4MzhmCOgPB7xb87f1CXjlA
2660 31JID2STTlLuFqXFWHyM8e
2661 2gMXnyrvIjhVBUZwvLZDMP
2662 2p8IUWQDrpjuFltbdgLOag
2664 3wYRLYuO1M88d8woWUIxct
2666 75dLEqanQTupF3dtP9exT5
2668 2o9LAypwGH4ctV0i9boo6d
2669 53cGGBxGPaRjP0HVUESyW5
2670 06scTb0zbkxYNgpAB3J9fN
2671 3o0nkjyy6eZL5lZ

3035 3Ofmpyhv5UAQ70mENzB277
3036 3FAJ6O0NOHQV8Mc5Ri6ENp
3041 1xK1Gg9SxG8fy2Ya373oqb
3042 4iCZ6Cr76For1TuHUQ90r8
3043 0awuNCj9gwJEM0seER22DW
3044 2JLVZYEMmqPWbUoAJAMKfm
3045 5cMoYjSCddCVVRUiVRv88q
3046 14JFKtBSqefxfMiKhq6JT8
3047 7vF3xkCMvZjAe2nTWY0uQZ
3048 472vIK1ldetTxRxG3ovaiY
3051 5hChuUMe5iHfjkXgzj1D4b
3052 6qBFSepqLCuh5tehehc1bd
3055 3aQem4jVGdhtg116TmJnHz
3057 4u4NyuceXP7Uzh7XFJKCr1
3058 5iMRhOHzze2CujxHiJ6cRL
3060 4eNOLmx8r2IJAVKvBBL1jv
3064 3Ic8V1CEjOY53OVpo2k1Ap
3065 6I3mqTwhRpn34SLVafSH7G
3066 6i4G85fdtDwK6UpQba337E
3068 2t96tGXVqmNT9KbbIXyEP1
3071 3zc8VZEpM1onYV4FWGdFvm
3074 5kHn6VJfMEThQYcG9AayaR
3075 5TWAIHYaOnYg4txfmCgon5
3076 4iXDn9pu5Q9sxv45vE8Lak
3077 53bZYxVbCYm6w1Gx8U64bz
3078 43PGPuHIlVOc04jrZVh9L6
3079 1SC5rEoYDGUK4NfG82494W
3081 6HU7h9RYOaPRFeh0R3UeAr
3082 6rTInqW3YECMkQsBEHw4sd
3083 6755DFiZ7CWe2WMdTGW5C1
3084 7uZqvqXzzYMKrexKxdcGmd
3088 7IhfjcaXuXCq20ZiAA7igK
3089 0GqaZ1L5KAXMNngAZbGvLf
3092 7FdUvDkaE24o3FPIWTvzv2
3093 2HYvkdq6lpFXkVz9Tfqi8r
3094 0NmuYnjETG3u3qx

### Search by title with artist traversal

In [87]:
CLIENT_ID = '6ced8cf12e0c470f9d9dd25a75d4ec26'
CLIENT_SECRET = '4ce366cebd7c42f2ac59bf96ca9f32b9'

AUTH_URL = 'https://accounts.spotify.com/api/token'

# POST
auth_response = requests.post(AUTH_URL, {
    'grant_type': 'client_credentials',
    'client_id': CLIENT_ID,
    'client_secret': CLIENT_SECRET,
})

# convert the response to JSON
auth_response_data = auth_response.json()

# save the access token
access_token = auth_response_data['access_token']
headers = {
    'Authorization': 'Bearer {token}'.format(token=access_token)
}

# base URL of all Spotify API endpoints
BASE_URL = 'https://api.spotify.com/v1/'
    
    
for i in hit_unique.loc[hit_unique['track_id'].isnull(),:].index.values:
    title = hit_unique['title'][i]
    # clean title
    #title = ''.join(c for c in hit_unique['title'][i] if c.isalnum() or c.isspace())
    #title = ' '.join(c for c in hit_unique['title'][i] if c.isalnum()) ## special format of titles
    
    # traverse artists for multi-artist songs
    for artist in re.split(', | & | Featuring ', hit_unique['artist'][i]):
        query = 'track:"%s" AND artist:%s&type=track&limit=1'%(title, artist)

        r = requests.get(BASE_URL + 'search?q=' + query, headers=headers).json()
        
        try:
            results = r['tracks']['items'][0]

            
            artists_i = getArtistNameList(results['artists'])

            for a in artists_i:
                # deal with middle name -> match only first and last name
                if (a.lower() in artist.lower()) or (artist.lower() in a.lower()) or\
                ((artist.split()[0].lower() in a.split()[0].lower()) and (artist.split()[-1].lower() in a.split()[-1].lower())):
                    print(i, results['id'])
                    hit_unique['track_id'][i] = results['id']
                    hit_unique['release_date'][i] = results['album']['release_date']
                    hit_unique['spotify.name'][i] = results['name']
                    hit_unique['spotify.artists'][i] = artists_i
                    break
            break
        except:
            pass

119 7sjwaBQ6M3aVigYfBjm1sL
126 3dvynJHFt37j6R8CIIEZh5
128 6VRhkROS2SZHGlp0pxndbJ
131 4grUGQeAfNs6yQOdgEkncM
134 4EfN6bixdOOgoLYR5C4cWo
139 4P6BuLsqtg5uISdE77ypI9
141 1lqKgoo11HKUoMpbouC7I4
147 3OdeZ1MhdHaRqAZLpowKWn
149 3hTfNt05rYtyMamwkxyHaZ
157 1LmN9SSHISbtp9LoaR5ZVJ
160 4d2hD63oUGU3s7L66JD0BE
165 0nq6sfr8z1R5KJ4XUk396e
167 4It00tEYLwpjQtAhqnjCPC
169 2bgW1SJqUjIkWcbxTPTIdy
173 0lHAMNU8RGiIObScrsRgmP
174 3Rh3zDYanZFHaeXFT4Asps
176 2lD4LsB3iQVnddkVM2RFAl
180 0V4l4GQhgnWQGtCWpvA7va
186 0mtPNQl5vePVwiI9Vykp7L
189 17zF2ISrDrQdaULA8cxReK
197 6j9iyrrmqWlQZ5SD1hSTaq
203 4wkQmYpAaMe41Rc3sYZ7Vz
204 6Nw1WHydftZy2joT2nTiCR
205 3r9bgSJlJz2zlevcBRYXko
207 77ZdAoTHYoF6Umo76HFD4m
211 0pwYLVXVknPSGUQb39cePC
213 4MevEW6tsWrpmzPKaqK0Uw
216 1cji6jI9LQ5zaStXrydl7F
221 1Mg8vLBYwizu7AuhYQO64V
233 32UIpKdyGzgTPTO8e5ge5c
237 6lgzJuBX3iqevLdtgNlyAU
240 1BvTqJLeB9ZeGU01gFzJbh
244 3Jv1XgMPVjdN5xGfnRJ20s
260 0332p5aiaQ6vyomilcGSy4
263 759b29QKmhaVSAuMp6AfWr
267 6XDSSxwOntlKPkE6wVzucb
269 0i5el041vd6nxrGEU8QRxy
2

1427 5HJId22hZ2IvFnvNSy6ZbE
1429 0nUGa8WQpQqg2vzJUprEo5
1431 7nD9nN3jord9wWcfW3Gkcm
1432 51Nwg5wCDDiqcTntgJUlLo
1435 1CDBn2Lm7E63XKQDsdCq6O
1437 7iaw359G2XT14uTfV9feip
1446 4ezAjoV01phDz1SqO6SzEU
1448 43uQtNJUmlEdJtjbz6GdGL
1454 7BKLCZ1jbUBVqRi2FVlTVw
1455 7zsXy7vlHdItvUSH8EwQss
1456 0lYBSQXN6rCTvUZvg9S0lU
1461 7AclP0W2jy6yrbvBiC4prQ
1466 5uDASfU19gDxSjW8cnCaBp
1470 4VJ7mnW4wjjIgPBM6KWvPu
1473 0MkruAJ99HAHNc36HKkcAK
1478 43bCmCI0nSgcT7QdMXY6LV
1479 7xHWNBFm6ObGEQPaUxHuKO
1486 0H8XeaJunhvpBdBFIYi6Sh
1488 5oyUurs28XRagLa8lgm6V3
1490 498ZVInMGDkmmNVpSWqHiZ
1493 6fujklziTHa8uoM5OQSfIo
1495 7MXVkk9YMctZqd1Srtv4MB
1500 2rizacJSyD9S1IQUxUxnsK
1506 0bHDsbWkPSJ01adXfObBzJ
1512 1oMu5b1F3GbswqbqzhwhTV
1523 3V8sM5OOG6YfDuDLa2IIYJ
1525 4ckuS4Nj4FZ7i3Def3Br8W
1534 4Km5HrUvYTaSUfiSGPJeQR
1538 5J5PXmMdQ2nh1lZOal8KmK
1539 2PC9Tl8SWVO2yhDDtJcrAi
1544 2q95XoeFGixx8b5LNF6Ey1
1557 3dhjNA0jGA8vHBQ1VdD6vV
1567 6mICuAdrwEjh6Y6lroV2Kg
1571 4h90qkbnW1Qq6pBhoPvwko
1572 3a1lNhkSLSkpJE4MSHpDu9
1575 5knuzwU65gJK7IF

2665 0dIz1I7IwX5ardG47AHxCZ
2667 1jaTQ3nqY3oAAYyCTbIvnM
2690 7eJMfftS33KTjuF7lTsMCx
2692 0nGXi46VcQQ56ZJR428MKS
2693 7idmHTAQQPUFqdjXkoooXD
2696 4nK5YrxbMGZstTLbvj6Gxw
2704 0IpnZchq8ek2A6pGEP2Qb1
2710 1Z0cZI0UzNbP9L8MzzGxqf
2714 7ytR5pFWmSjzHJIeQkgog4
2715 0oT9ElXYSxvnOOagP9efDq
2720 5gNOINI5dXZVom2b36LMOd
2723 3oHiR89Y8gn6xt3YGAAzFj
2724 16oi4OiKmkO51l60F5R9kR
2731 5rFGVn8fCFjnRwCebW0XbS
2734 2kJwzbxV2ppxnQoYw4GLBZ
2737 5o6dFqafETxZNqUFt9ZqAC
2749 1C8ux1YWLMHCenYJFiIV4Z
2757 3nS9a01VvXHQriLqJYwRqG
2759 5LHHKZOwV8XW4LJP2C64mw
2761 2QqJTIlGKRLJC3onkavYEz
2763 4YhJ3a6kBPGJWX4Ek77sDd
2765 4wVOKKEHUJxHCFFNUWDn0B
2766 02mYw61RoS1vXvkD9Q5dpN
2768 6mtjo5kIHKlboGE84lf1FJ
2771 77fvnWALKJR0ws1VUx09CY
2778 4kfeRwpq5KUaqTkgi4TbDF
2780 740kQ8xwSGloumdiKR5Cwz
2784 65jw6GhwQW3Db55PxlPEpS
2789 62aP9fBQKYKxi7PDXwcUAS
2800 4EivmOT13NMpNSfTKn9p4s
2806 5FmvaZGd6fulojSToozLRD
2811 6HM2tjTLqJi0poAkCdnNTB
2815 6EDO9iiTtwNv6waLwa1UUq
2816 35RJhm1pEovTBwnNR0zWad
2819 0PvFJmanyNQMseIFrU708S
2820 1H7KnK26kc1Yyel

### Search by title(cleaned) and traverse artists

In [88]:
CLIENT_ID = '6ced8cf12e0c470f9d9dd25a75d4ec26'
CLIENT_SECRET = '4ce366cebd7c42f2ac59bf96ca9f32b9'

AUTH_URL = 'https://accounts.spotify.com/api/token'

# POST
auth_response = requests.post(AUTH_URL, {
    'grant_type': 'client_credentials',
    'client_id': CLIENT_ID,
    'client_secret': CLIENT_SECRET,
})

# convert the response to JSON
auth_response_data = auth_response.json()

# save the access token
access_token = auth_response_data['access_token']
headers = {
    'Authorization': 'Bearer {token}'.format(token=access_token)
}

# base URL of all Spotify API endpoints
BASE_URL = 'https://api.spotify.com/v1/'
    
    
for i in hit_unique.loc[hit_unique['track_id'].isnull(),:].index.values:
    #title = hit_unique['title'][i]
    # clean title
    title = ''.join(c for c in hit_unique['title'][i] if c.isalnum() or c.isspace())
    ##title = ' '.join(c for c in hit_unique['title'][i] if c.isalnum()) ## special format of titles
    
    # traverse artists for multi-artist songs
    for artist in re.split(', | & | Featuring | X ', hit_unique['artist'][i]):
        # direct search by title and artist
        query = 'track:"%s" AND artist:%s&type=track&limit=1'%(title, artist)

        r = requests.get(BASE_URL + 'search?q=' + query, headers=headers).json()
        
        try:
            results = r['tracks']['items'][0]
        
            artists_i = getArtistNameList(results['artists'])

            for a in artists_i:
                # deal with middle name -> match only first and last name
                if (a.lower() in artist.lower()) or (artist.lower() in a.lower()) or\
                ((artist.split()[0].lower() in a.split()[0].lower()) and (artist.split()[-1].lower() in a.split()[-1].lower())):
                    print(i, results['id'])
                    hit_unique['track_id'][i] = results['id']
                    hit_unique['release_date'][i] = results['album']['release_date']
                    hit_unique['spotify.name'][i] = results['name']
                    hit_unique['spotify.artists'][i] = artists_i
                    break
            break
        except:
            pass

7 6D60klaHqbCl9ySc8VcRss
11 5HQVUIKwCEXpe7JIHyY734
25 0cV4xwUA4ue2deqq4CZFko
39 42EMFjvFtOki8MA0JNZqVK
56 3mTpegrOwRn0oJjv4TSbEE
70 4GwbSrCJFRze2tK4H12Zew
71 53QF56cjZA9RTuuMZDrSA6
90 0z9UVN8VBHJ9HdfYsOuuNf
96 4HRa9ZWcmZ8JD7UySPu0wT
121 0prF3F4FoKvxBObaeXLVHv
130 4WnY4W5KrqglCdIus8X9Bo
153 1NJoVAf6HvRbAMH1Py2XmN
226 1YTO55iUyFqx1QgcWXArLB
241 18VfHIPaCo0s6FTi1Fnp6T
280 2V65y3PX4DkRhy1djlxd9p
282 2ikkgfIIM9kGKZCXzXnTpj
306 6Vh03bkEfXqekWp7Y1UBRb
331 3Tu7uWBecS6GsLsL8UONKn
337 0NG5gJzPHjYeQmqcmqfopN
342 6MVdz8I7oYm86MRzZiOZa8
343 6J5sxraPPZ4b0CVOGAgpXj
344 2iJuuzV8P9Yz0VSurttIV5
367 6vt0I1cw1YmAIKDJvHVIM5
378 6PQ0WLZnT479J7Q1TGIb3n
405 5sdhqXSfvyDgU6fiDJTP4f
411 3bidbhpOYeV4knp8AIu8Xn
413 01TuObJVd7owWchVRuQbQw
417 4sSUYUpB5epI7xTi5wfpOI
438 6LoQHIo74tOzQ8EsLEkhgF
459 1ahAuX3F6tDdNRfMdiLsv6
474 1lNl7zRexUwxQ20OKONF3A
475 5vTPxzm4h2bY9rYyVrGEU5
477 2y4lAQpi5VTNLu2ldeTdUH
480 3Qe097eJNp9MfpT2PieEw0
485 00QyLmjxaSEE8qIZQjBXBj
501 0UAmPUdpTRq8w54B3H8TzY
517 3c7MNgwEBQTnrOWLjU1DHq
537 2aSC2xh

### Null

In [90]:
#hit_unique[hit_unique['track_id'].isnull()].to_csv('billboard_null.csv')

#hit_null = pd.read_csv('billboard_null.csv')

#hit_unique.iloc[hit_null['Unnamed: 0'], 4] = hit_null.iloc[:,5]

### Title discrepancy

In [ ]:
#title_dif_idx = []
#for i in hit_unique.loc[~hit_unique['spotify.name'].isnull()].index.values:
    #if str(hit_unique['title'][i]).lower() not in str(hit_unique['spotify.name'][i]).lower():
        #title_dif_idx.append(i)

#title_dif = hit_unique.iloc[title_dif_idx]

#new_title = pd.read_excel('title.xlsx')
#hit_unique.iloc[new_title['Unnamed: 0'], 4] = new_title.iloc[:,5]

In [98]:
hitunique = hit_unique.copy()

### Duplicated ID

In [ ]:
#dup = hit_unique[hit_unique.duplicated(['track_id'],keep=False)]
#dup = dup.loc[~dup['track_id'].isnull()].sort_values('track_id')

#new_dup = pd.read_excel('dup.xlsx')
#hit_unique.iloc[new_dup['Unnamed: 0'], 4] = new_dup.iloc[:,5]

In [102]:
#hit_unique.dropna(subset=['track_id']).to_csv('billboard_ID.csv', index=False)

## Get track information

In [103]:
hit_unique = pd.read_csv('billboard_ID_filled.csv').iloc[:,0:5]
hit_unique

,title,artist,date,hit,track_id
0,Set Fire To The Rain,Adele,2012-02-04,1,73CMRj62VK8nUS4ezD2wvi
1,We Found Love,Rihanna Featuring Calvin Harris,2012-02-04,1,6qn9YLKt13AGvpq9jfO8py
2,Good Feeling,Flo Rida,2012-02-04,1,2LEF1A8DOZ9wRYikWgVlZ8
3,Turn Me On,David Guetta Featuring Nicki Minaj,2012-02-04,1,6JOlNkT0QdHeZB0wPbI9IR
4,The One That Got Away,Katy Perry,2012-02-04,1,6hkOqJ5mE093AQf2lbZnsG
...,...,...,...,...,...
3266,7am On Bridle Path,Drake,2021-10-02,1,42m3eP1JJhtzffal9B136J
3267,Off The Grid,Kanye West,2021-10-02,1,6LNoArVBBVZzUTUiAX2aKO
3268,In Da Getto,J Balvin & Skrillex,2021-10-02,1,63aj87TQG6F3RVO5nbG2VQ
3269,Am I Dreaming,Lil Nas X Featuring Miley Cyrus,2021-10-02,1,6isTQfKXhNO3EyJd9mSxx8


In [110]:
# initialize track_info dataframe
track_info = pd.DataFrame(columns=['artists', 'available_markets', 'disc_number', 'duration_ms',
       'explicit', 'href', 'id', 'is_local', 'name', 'popularity',
       'preview_url', 'track_number', 'type', 'uri', 'album.album_type',
       'album.artists', 'album.available_markets',
       'album.external_urls.spotify', 'album.href', 'album.id', 'album.images',
       'album.name', 'album.release_date', 'album.release_date_precision',
       'album.total_tracks', 'album.type', 'album.uri', 'external_ids.isrc',
       'external_urls.spotify'])

for i in range(len(track_info), len(hit_unique), 50):
    if not i+50 > len(hit_unique):
        ids = ','.join(hit_unique['track_id'][i:i+50])
    else:
        ids = ','.join(hit_unique['track_id'][i:])
        
    r = requests.get(BASE_URL + 'tracks?ids=' + ids , headers=headers)
    
    track_info = pd.concat([track_info, pd.json_normalize(r.json()['tracks'])],
                               ignore_index=True)

In [ ]:
def getArtistName(artists_list):
    artists = []
    for a in artists_list:
        artists.append(a['name'])
    return ', '.join(artists)

track_info_clean = track_info[['name','artists','id','album.release_date']]

for i in range(len(track_info_clean)):
    track_info_clean['artists'][i] = getArtistName(track_info_clean['artists'][i])

In [ ]:
track_info_clean

In [ ]:
hit_unique.merge(track_info_clean, left_on='track_id', right_on='id').to_csv('billboard_ID_spotify.csv', index=False)